## To decide number of hidden layers and neurons for a NeuralNetwork

In [2]:
import tensorflow

In [7]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [8]:
df=pd.read_csv('Real_Combine.csv')

In [9]:
df.head()

#About Air Quality prediction

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [10]:
X=df.iloc[:,:-1]#Independent Feature
Y=df.iloc[:,-1]#Dependent Feature

### HyperPArameters

1. no. of hidden layers
2. no. of neurons
3. learning rate

In [24]:
def build_model(hp): #hp is the instance of the model we are creating
    model=keras.Sequential()#Stack of layers
    for i in range(hp.Int('num_layers',2,20)): #creating a neural network which can have layers between 2 to 20
                                                #hp.Int means try different values
        model.add(layers.Dense(units=hp.Int('units_'+str(i), #Dense adds neurons to a layer(between 32-512)
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                              activation='relu')) #activation function(works well in hidden layers)
    model.add(layers.Dense(1,activation='linear')) #Output layer with 1 neuron
    model.compile(
        optimizer=keras.optimizers.legacy.Adam(
        hp.Choice('learning_rate',[1e-2,1e-3,1e-4])), #hp.choice means choose from the given
    loss='mean_absolute_error',
    metrics=['mean_absolute_error'])
    return model

In [36]:
tuner=RandomSearch(
    build_model, #the function
    objective='val_mean_absolute_error',
    max_trials=5, #maximum number of trials
    max_retries_per_trial=3, #total 15 times
    max_consecutive_failed_trials=1,
    directory='project', #save the weights
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Tuner from project/Air Quality Index/tuner0.json


INFO:tensorflow:Reloading Tuner from project/Air Quality Index/tuner0.json


In [37]:
tuner.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [39]:
tuner.search(X_train,y_train,epochs=50,validation_data=(X_test,y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
18                |15                |num_layers
64                |64                |units_0
288               |224               |units_1
0.001             |0.001             |learning_rate
480               |32                |units_2
288               |32                |units_3
224               |32                |units_4
320               |32                |units_5
224               |32                |units_6
96                |32                |units_7
128               |32                |units_8
128               |32                |units_9
32                |32                |units_10
96                |32                |units_11
224               |32                |units_12
32                |32                |units_13
288               |32                |units_14
32                |None              |units_15
32                |None              |units_16
32                |None       

Epoch 46/50
23/23 [==============================] - 0s 22ms/step - loss: 46.0904 - mean_absolute_error: 46.0904 - val_loss: nan - val_mean_absolute_error: nan
Epoch 47/50
23/23 [==============================] - 1s 22ms/step - loss: 44.7423 - mean_absolute_error: 44.7423 - val_loss: nan - val_mean_absolute_error: nan
Epoch 48/50
23/23 [==============================] - 1s 22ms/step - loss: 44.6025 - mean_absolute_error: 44.6025 - val_loss: nan - val_mean_absolute_error: nan
Epoch 49/50
23/23 [==============================] - 0s 21ms/step - loss: 46.7412 - mean_absolute_error: 46.7412 - val_loss: nan - val_mean_absolute_error: nan
Epoch 50/50
23/23 [==============================] - 1s 23ms/step - loss: 44.3194 - mean_absolute_error: 44.3194 - val_loss: nan - val_mean_absolute_error: nan


/Users/shlok/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras_tuner/engine/metrics_tracking.py:110: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


TypeError: can only concatenate str (not "NoneType") to str

In [40]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 0 summary
Hyperparameters:
num_layers: 15
units_0: 64
units_1: 224
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score: nan

Trial 1 summary
Hyperparameters:
num_layers: 5
units_0: 32
units_1: 384
learning_rate: 0.01
units_2: 512
units_3: 32
units_4: 160
units_5: 128
units_6: 416
units_7: 480
units_8: 480
units_9: 224
units_10: 416
units_11: 480
units_12: 192
units_13: 192
units_14: 416
Score: nan

Trial 2 summary
Hyperparameters:
num_layers: 18
units_0: 64
units_1: 288
learning_rate: 0.001
units_2: 480
units_3: 288
units_4: 224
units_5: 320
units_6: 224
units_7: 96
units_8: 128
units_9: 128
units_10: 32
units_11: 96
units_12: 224
units_13: 32
units_14: 288
units_15: 32
units_16: 32
units_17: 32
Score: nan
